In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os

In [2]:
print("Tensorflow version: {}".format(tf.__version__))

Tensorflow version: 2.2.0


In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
keras = tf.keras
layers = tf.keras.layers

In [5]:
train_image_path=glob.glob('./data/dc/training_set/*/*.jpg')
test_image_path=glob.glob('./data/dc/test_set/*/*.jpg')

In [6]:
train_image_label=[int(os.path.basename(p).split('.')[0]=='cat') for p in train_image_path]
test_image_label=[int(os.path.basename(p).split('.')[0]=='cat') for p in test_image_path]

In [7]:
def load_preprocess_image(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[256,256])
    image = tf.cast(image, tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [8]:
train_image_ds=tf.data.Dataset.from_tensor_slices((train_image_path, train_image_label))
test_image_ds=tf.data.Dataset.from_tensor_slices((test_image_path, test_image_label))

In [9]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
train_image_ds=train_image_ds.map(load_preprocess_image,num_parallel_calls=AUTOTUNE)
test_image_ds=test_image_ds.map(load_preprocess_image,num_parallel_calls=AUTOTUNE)

In [10]:
train_image_ds

<ParallelMapDataset shapes: ((256, 256, 3), (1,)), types: (tf.float32, tf.int32)>

In [11]:
BATCH_SIZE=32
train_count = len(train_image_path)
test_count = len(test_image_path)

train_image_ds=train_image_ds.shuffle(train_count).batch(BATCH_SIZE)
train_image_ds=train_image_ds.prefetch(AUTOTUNE).repeat()

test_image_ds=test_image_ds.batch(BATCH_SIZE)
test_image_ds=test_image_ds.prefetch(AUTOTUNE).repeat()

In [12]:
covn_base = keras.applications.VGG16(weights='imagenet',include_top=False)

In [13]:
covn_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [14]:
model=keras.Sequential()
model.add(covn_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 14,977,857
Non-trainable params: 0
_________________________________________________________________


In [16]:
covn_base.trainable=False

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 263,169
Non-trainable params: 14,714,688
_________________________________________________________________


In [18]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
             loss='binary_crossentropy',
             metrics=['acc'])

In [19]:
with tf.device('/cpu:0'):
    history=model.fit(
        train_image_ds,
        steps_per_epoch=train_count//BATCH_SIZE,
        epochs=15,
        validation_data=test_image_ds,
        validation_steps=test_count//BATCH_SIZE)

Epoch 1/15
250/250 [==============================] - 690s 3s/step - loss: 0.3363 - acc: 0.8541 - val_loss: 0.2249 - val_acc: 0.9062
Epoch 2/15
250/250 [==============================] - 647s 3s/step - loss: 0.2090 - acc: 0.9143 - val_loss: 0.2133 - val_acc: 0.9082
Epoch 3/15
250/250 [==============================] - 627s 3s/step - loss: 0.1852 - acc: 0.9197 - val_loss: 0.1893 - val_acc: 0.9157
Epoch 4/15
250/250 [==============================] - 626s 3s/step - loss: 0.1794 - acc: 0.9261 - val_loss: 0.2102 - val_acc: 0.8983
Epoch 5/15
250/250 [==============================] - 626s 3s/step - loss: 0.1735 - acc: 0.9269 - val_loss: 0.1827 - val_acc: 0.9187
Epoch 6/15
250/250 [==============================] - 626s 3s/step - loss: 0.1655 - acc: 0.9328 - val_loss: 0.1691 - val_acc: 0.9266
Epoch 7/15
250/250 [==============================] - 626s 3s/step - loss: 0.1519 - acc: 0.9385 - val_loss: 0.1774 - val_acc: 0.9241
Epoch 8/15
250/250 [==============================] - 626s 3s/step - 